In [1]:
import stim

# 9-qubit Shor code: blocks [0,1,2], [3,4,5], [6,7,8]
# This prepares |0_L>, applies DEPOLARIZE1(0.02) right after init,
# then measures all stabilizers and the logical Z.

c = stim.Circuit("""
# --- Initialize physical qubits to |0> ---
R 0 1 2 3 4 5 6 7 8

# --- Encode |0_L> ---
# Create GHZ within each of the 3 blocks: (|000>+|111>)/sqrt(2)
H 0 3 6
CNOT 0 1
CNOT 0 2
CNOT 3 4
CNOT 3 5
CNOT 6 7
CNOT 6 8

# --- Error layer right after initialization (of the logical state) ---
DEPOLARIZE1(0.02) 0 1 2 3 4 5 6 7 8

# --- Measure Z-type stabilizers (within each 3-qubit block) ---
# Block 1
MPP Z0*Z1
DETECTOR rec[-1]
MPP Z1*Z2
DETECTOR rec[-1]
# Block 2
MPP Z3*Z4
DETECTOR rec[-1]
MPP Z4*Z5
DETECTOR rec[-1]
# Block 3
MPP Z6*Z7
DETECTOR rec[-1]
MPP Z7*Z8
DETECTOR rec[-1]

# --- Measure X-type stabilizers (across blocks) ---
MPP X0*X1*X2*X3*X4*X5
DETECTOR rec[-1]
MPP X3*X4*X5*X6*X7*X8
DETECTOR rec[-1]

# --- Logical Z readout ---
MPP Z0*Z3*Z6
OBSERVABLE_INCLUDE(0) rec[-1]
""")

# Example: sample detection events and the logical observable.
shots = 10000
det_samp = c.compile_detector_sampler().sample(shots=shots)
obs_samp = c.compile_observable_sampler().sample(shots=shots)

print(c)  # show the circuit
print("Detector samples shape:", det_samp.shape)  # (shots, 8 stabilizers)
print("Observable samples shape:", obs_samp.shape)  # (shots, 1 logical Z)
print("Logical Z error rate estimate:", obs_samp.mean())


AttributeError: 'stim._stim_sse2.Circuit' object has no attribute 'compile_observable_sampler'